In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df = pd.read_csv("/content/Airline_dataset.csv",index_col=0)

In [ ]:
df.shape

(1204825, 13)

In [ ]:
df.isnull().sum()

,0
FL_DATE,0
AIRLINE_ID,0
TAIL_NUM,0
FLIGHT_NUM,0
ORIGIN_SEQ_ID,0
ORIGIN_AIRPORT,0
DEST_SEQ_ID,0
DEST_AIRPORT,0
DEP_TIME,0
DEP_DELAY,578


In [ ]:
df.dropna(inplace=True)

In [ ]:
df.isnull().sum()

,0
FL_DATE,0
AIRLINE_ID,0
TAIL_NUM,0
FLIGHT_NUM,0
ORIGIN_SEQ_ID,0
ORIGIN_AIRPORT,0
DEST_SEQ_ID,0
DEST_AIRPORT,0
DEP_TIME,0
DEP_DELAY,0


In [ ]:
df.head()

,FL_DATE,AIRLINE_ID,TAIL_NUM,FLIGHT_NUM,ORIGIN_SEQ_ID,ORIGIN_AIRPORT,DEST_SEQ_ID,DEST_AIRPORT,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,WEATHER_DELAY
0,8/1/18,19805.0,N956AN,1587.0,1247805.0,JFK,1410702.0,PHX,1649.0,9.0,2006.0,44.0,0.000000
1,8/1/18,19805.0,N973AN,1588.0,1410702.0,PHX,1161802.0,EWR,1541.0,29.0,2350.0,53.0,0.000000
2,8/1/18,19805.0,N9006,1590.0,1104205.0,CLE,1129806.0,DFW,741.0,-3.0,938.0,-2.0,4.030195
3,8/1/18,19805.0,N870NN,1591.0,1484306.0,SJU,1129806.0,DFW,944.0,44.0,1347.0,43.0,0.000000
4,8/1/18,19805.0,N9023N,1593.0,1042302.0,AUS,1330303.0,MIA,556.0,-4.0,951.0,-2.0,4.030195


In [ ]:
df.to_csv('cleaned_data_final.csv',index=False)

# **FLIGHT ASSIGNMENT OPTIMIZATION USING LINEAR PROGRAMMING**

**The aim here is to assign aircrafts to the scheduled flights in a way that the total delay is reduced, while ensuring that ;**


1.   No overlap for the flight schedules
2.   Minimizing total weather delay
3.   Ensuring Aircraft routes are feasible


In [ ]:
df = pd.read_csv("/content/cleaned_data_final.csv")

In [ ]:
df['DEP_TIME'] = pd.to_numeric(df['DEP_TIME'])
df['ARR_TIME'] = pd.to_numeric(df['ARR_TIME'])

In [ ]:
#Fixing the time to be in 12:12 format rather than 1212
from datetime import datetime, timedelta
def to_datetime(time_val):
    time_val = int(time_val)
    hour = time_val // 100
    minute = time_val % 100
    return timedelta(hours=hour, minutes=minute)

df['dep_dt'] = df['DEP_TIME'].apply(to_datetime)
df['arr_dt'] = df['ARR_TIME'].apply(to_datetime)

In [ ]:
df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

In [ ]:
#since the dataset has 700,000 rows, i want to reduce it.
#lets only focus on '2018-08-01 and aim is to keep number of rows about 2000 to avoid heavy computation'
df_lp = df[df['FL_DATE']=='2018-08-01'].sample(n=200, random_state=2)

In [ ]:
df_lp.shape

(200, 15)

In [ ]:
!pip install pulp

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 83.1 MB/s eta 0:00:00


In [ ]:
import pulp
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary
from itertools import combinations

In [ ]:
#Deciding the variables
flights = df_lp.index.tolist()
x = pulp.LpVariable.dicts("Flight",flights,cat='Binary')

#Initializing the problem
prob = pulp.LpProblem("Flight_Scheduling_Min_Weather_Delay", LpMinimize)

#Defining the Objective
prob += pulp.lpSum([x[i] * df_lp.loc[i,'WEATHER_DELAY'] + (1-x[i]) * 100 for i in flights])

#Constraint that we are checking is to ensure that no flights are overlapping
for tail_num in df_lp['TAIL_NUM'].unique()[:3]:  # Limit to first 3 tail numbers for debugging
    same_tail_num = df_lp[df['TAIL_NUM'] == tail_num].sort_values(by='DEP_TIME')
    flights_list = same_tail_num.index.tolist()
    for i_idx in range(len(flights_list)):
        i = flights_list[i_idx]
        arr_i = df_lp.loc[i, 'ARR_TIME']
        for j_idx in range(i_idx + 1, len(flights_list)):
            j = flights_list[j_idx]
            dep_j = df_lp.loc[j, 'DEP_TIME']
            if arr_i + 10 <= dep_j:  # Relaxed constraint (10-minute buffer)
                prob += x[i] + x[j] <= 1

prob.solve()

1

In [ ]:
print("Status:", pulp.LpStatus[prob.status])
selected_flights = [i for i in flights if x[i].varValue == 1]
print("Flights Selected:", selected_flights)
print("Total Weather Delay:", sum(df.loc[i, 'WEATHER_DELAY'] for i in selected_flights))

Status: Optimal
Flights Selected: [15847, 10831, 21927, 2059, 11744, 2710, 6744, 4455, 4287, 418, 4778, 12600, 12952, 21213, 8760, 6873, 3816, 10595, 14344, 2919, 20021, 14378, 20310, 5547, 10875, 20271, 22620, 12840, 8495, 22542, 2436, 10397, 15429, 1462, 1975, 7849, 20883, 20632, 22654, 15509, 8357, 18080, 6499, 130, 10865, 1736, 14753, 15444, 21845, 17235, 5615, 17712, 8144, 1295, 11075, 10177, 14306, 21469, 2247, 8616, 15267, 17696, 9632, 12780, 7229, 11723, 3035, 6237, 10221, 10173, 1963, 21229, 2813, 14472, 9812, 9327, 7574, 14230, 4610, 14478, 10882, 13378, 10469, 5963, 22945, 16437, 21352, 9236, 9671, 21522, 8986, 2367, 15438, 2929, 6824, 173, 12205, 7265, 22974, 17518, 12398, 4737, 15961, 1591, 15592, 19595, 8432, 15070, 18792, 62, 15171, 20912, 2440, 9587, 20431, 5943, 7897, 3459, 15819, 22604, 21496, 19404, 13921, 3636, 19955, 18745, 16440, 2032, 14756, 11440, 17799, 11490, 1650, 437, 22421, 16105, 7888, 577, 13335, 5210, 3618, 17293, 10432, 11287, 6737, 4467, 1032, 16743, 2

# **CREW SELECTION SYSTEM USING MILP WITH TWO DIFFERENT VARIABLES**

Objective: Minimizing the total usage of crews


1.   One crew per flight
2.   No overbooking/ overlapping of crew members
3.   Have to ensure that Link usage is done so that the count of crew is intact



In [ ]:
import pulp
from pulp import LpProblem, LpVariable, LpMinimize, lpSum, LpBinary
from itertools import product

In [ ]:
#dEFINING CREW AND FLIGHT IDs to ensure that the MILP problem has the desired variables
#we are assuming that we have 20 crew member,
#and the flight id would be the unique case for each and every data point in the dataset.

crew_num = 20
crew_id = list(range(crew_num))
df_lp['Flight_ID'] = df_lp.index
flight_id = df_lp['Flight_ID'].tolist()

In [ ]:
len(set(flight_id)) #shows we have unique flight IDs

200

In [ ]:
df_lp['DEP_TIME']

,DEP_TIME
15847,1801.0
10831,1115.0
21927,1630.0
2059,1532.0
11744,2211.0
...,...
3677,1221.0
8420,2303.0
13049,1647.0
9951,1030.0


In [ ]:
#Fixing the Times to minutes after midnight for easy comparison
def time_to_minutes(time):
    hours = time // 100  # Get the hour part
    minutes = time % 100  # Get the minute part
    return hours * 60 + minutes  # Convert to total minutes

# Apply the conversion to your dataframe
df_lp['DEP_TIME_minutes'] = df_lp['DEP_TIME'].apply(time_to_minutes)
df_lp['ARR_TIME_minutes'] = df_lp['ARR_TIME'].apply(time_to_minutes)


#variable 1: Assign crew c to flight f
assign = pulp.LpVariable.dicts("Assign", (crew_id, flight_id), cat='Binary')

#variable 2: The crew must be assigned value 1 if selected, else 0
used_crew = pulp.LpVariable.dicts("crew_used", crew_id, cat='Binary')

#defining the problem statement; minimizing the total number of crew members
prob = pulp.LpProblem("Crew_Assignment_Problem", LpMinimize)

#Minimizing this
prob += pulp.lpSum(used_crew[c] for c in crew_id)

#overlap checker
schedule = {f: {'dep': df_lp.loc[f, 'DEP_TIME_minutes'], 'arr': df_lp.loc[f, 'ARR_TIME_minutes']} for f in flight_id}

def checker(f1, f2):
    # Skip comparison if flights are too far apart
    if abs(schedule[f1]['dep'] - schedule[f2]['dep']) > 30:
        return False
    return schedule[f1]['arr'] > schedule[f2]['dep'] and schedule[f2]['arr'] > schedule[f1]['dep']

#Constraint 1; only one crew member per flight
for f in flight_id:
  prob += pulp.lpSum(assign[c][f] for c in crew_id) == 1

#Constraint 2; no overlapping
for c in crew_id:
  for f1,f2 in product(flight_id,repeat=2):
    if f1 >= f2: continue
    if checker(df_lp[df_lp['Flight_ID'] == f1].index[0],df_lp[df_lp['Flight_ID'] == f2].index[0]):
      prob += assign[c][f1] + assign[c][f2] <= 1

#Linking usage; this is done to ensure that if a crew is booked, we count it as 1 and if the crew is not booked on any flight, the count is 0
for c in crew_id:
  for f in flight_id:
    prob += assign[c][f] <= used_crew[c]

#Solving the problem
prob.solve()

1

In [ ]:
# Output
print("Status:", pulp.LpStatus[prob.status])
print("Total number of crews:", crew_num)
print("Minimum number of crews used:", sum(pulp.value(used_crew[c]) for c in crew_id))

# Show assignments
crew_assignments = []
for c in crew_id:
    for f in flight_id:
        if pulp.value(assign[c][f]) == 1:
            crew_assignments.append((f, c))

df_assignments = pd.DataFrame(crew_assignments, columns=['FLIGHT_ID', 'CREW_ID'])
df_assignments

Status: Optimal
Total number of crews: 20
Minimum number of crews used: 13.0


,FLIGHT_ID,CREW_ID
0,10595,0
1,5547,0
2,8495,0
3,1462,0
4,21845,0
...,...,...
195,4467,17
196,10840,17
197,15312,17
198,9865,17


# **CREW MANAGEMENT WITH MDP**

In [ ]:
#if need to use again
# def time_to_minutes(time):
#     hours = time // 100  # Get the hour part
#     minutes = time % 100  # Get the minute part
#     return hours * 60 + minutes  # Convert to total minutes

# # Apply the conversion to your dataframe
# df_lp['DEP_TIME_minutes'] = df_lp['DEP_TIME'].apply(time_to_minutes)
# df_lp['ARR_TIME_minutes'] = df_lp['ARR_TIME'].apply(time_to_minutes)

In [ ]:
df_lp['DEP_DELAY']

,DEP_DELAY
15847,6.0
10831,-5.0
21927,1.0
2059,1.0
11744,71.0
...,...
3677,56.0
8420,193.0
13049,17.0
9951,-15.0


In [ ]:
class mdpCrewManagement:
  def __init__(self, flight_df, crew_list):
    self.flight_df = flight_df
    self.crew_list = crew_list
    self.crew_availability = {crew: True for crew in self.crew_list}
    self.current_flight_idx = 0  # new: to track which flight we're at
    self.state = self.initialize_state()

  def initialize_state(self):
    first_flight = self.flight_df.iloc[self.current_flight_idx]
    first_crew = self.crew_list[0]
    return (first_crew, first_flight['Flight_ID'], first_flight['DEP_TIME_minutes'])

  def step(self, state, action):
    crew, Flight_ID, time = state

    reward = 0
    next_state = state  # default

    if self.current_flight_idx >= len(self.flight_df):
        return state, 0  # No more flights, stay in same state

    flight = self.flight_df.iloc[self.current_flight_idx]
    dep_delay = max(flight['DEP_DELAY'], 0)
    arr_delay = max(flight['ARR_DELAY'], 0)

    if action == 'assign':
        if self.crew_availability[crew]:
            self.crew_availability[crew] = False
            reward = 1 if dep_delay == 0 and arr_delay == 0 else -10
        else:
            reward = -10
    elif action == 'wait':
        reward = -5

    #Always move to next flight (MDP progresses)
    self.current_flight_idx += 1

    if self.current_flight_idx < len(self.flight_df):
        next_flight = self.flight_df.iloc[self.current_flight_idx]
        next_state = (str(crew), int(next_flight['Flight_ID']), float(next_flight['DEP_TIME_minutes']))

    return next_state, reward

  def is_terminal(self, state):
    return self.current_flight_idx >= len(self.flight_df) - 1

In [ ]:
import random

crew_list = ['crew1','crew2','crew3','crew4','crew5']
mdp = mdpCrewManagement(df_lp, crew_list)

# Run the MDP for a few steps (simulating some crew assignments)

actions = ['assign', 'wait']  # Possible actions
state = mdp.state
for step_num in range(5):
    action = random.choice(actions)
    state = (crew_list[step_num % len(crew_list)], state[1], state[2])  # update crew each step
    next_state, reward = mdp.step(state, action)

    print(f"Step {step_num + 1}:")
    print(f"Action: {action}")
    print(f"Next State: {next_state}")
    print(f"Reward: {reward}\n")

    state = next_state

Step 1:
Action: assign
Next State: ('crew1', 10831, 675.0)
Reward: -10

Step 2:
Action: assign
Next State: ('crew2', 21927, 990.0)
Reward: 1

Step 3:
Action: wait
Next State: ('crew3', 2059, 932.0)
Reward: -5

Step 4:
Action: assign
Next State: ('crew4', 11744, 1331.0)
Reward: -10

Step 5:
Action: assign
Next State: ('crew5', 2710, 1131.0)
Reward: -10

